#### Імпортуємо всі бібліотеки

In [1]:
import datetime
import os
import urllib
import urllib.request
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os

print("Setup complete!")

Setup complete!


In [ ]:
folder_path = 'download'
if not os.path.exists(folder_path):
    os.makedirs(folder_path, exist_ok=True)
print(f"Folder '{folder_path}' was successfully created or already exists.")

#### Функція для завантаження даних

In [ ]:
def download_data(province_id, year1=1981, year2=2024):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year1}&year2={year2}&type=Mean"
    response = urllib.request.urlopen(url)
    
    if response.status == 200:  
        current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
        filename = f'vhi_id__{province_id}__{current_datetime}.csv'
        with open(f'download/{filename}', 'wb') as out: 
            out.write(response.read())
        print(f"VHI is downloaded for province ID {province_id} into {filename}")
    else:
        print(f"Failed to download data for province ID {province_id}. HTTP status code: {response.status}")

#### Завантаження тестових структуровних даних за номером області

In [ ]:
for i in range(1, 28):
    download_data(i)
print("The loading of test structural data was successful")

#### Відповідність id регіону та його назви

In [ ]:
reg_id_name = {
    1: "Cherkasy",
    2: "Chernihiv",
    3: "Chernivtsi",
    4: "Crimea",
    5: "Dnipropetrovs'k",
    6: "Donets'k",
    7: "Ivano-Frankivs'k",
    8: "Kharkiv",
    9: "Kherson",
    10: "Khmel'nyts'kyy",
    11: "Kiev",
    12: "Kiev City",
    13: "Kirovohrad",
    14: "Luhans'k",
    15: "L'viv",
    16: "Mykolayiv",
    17: "Odessa",
    18: "Poltava",
    19: "Rivne",
    20: "Sevastopol",
    21: "Sumy",
    22: "Ternopil'",
    23: "Transcarpathia",
    24: "Vinnytsya",
    25: "Volyn",
    26: "Zaporizhzhya",
    27: "Zhytomyr",
}

#### Функція ощищення даних і створення загального DataFrame

In [ ]:
import glob
import pandas as pd
def create_data_frame(folder_path1):

    csv_files = glob.glob(folder_path1 + "/*.csv")

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    frames = []

    for file in csv_files:
        region_id1 = int(file.split('__')[1]) 
        df = pd.read_csv(file, header=1, names=headers)  
        df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
        df=df.drop(df.index[-1])
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df.insert(0, 'region_id', region_id1, True)
        frames.append(df)
        
    result = pd.concat(frames).drop_duplicates().reset_index(drop=True)
    return result

In [ ]:
result_df = create_data_frame('download')
print(result_df)

####  Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [ ]:
def region_year_analysis(region_id, years=("1982", "2024")):
    result_df['Year'] = pd.to_numeric(result_df['Year'])

    df2 = result_df[(result_df["Year"].between(int(years[0]), int(years[1]))) & (result_df['region_id'] == region_id)]
    region_name = reg_id_name[region_id]
    vhi_max_reg = df2["VHI"].max()
    vhi_min_reg = df2["VHI"].min()
    print(f"[+] {region_name}: min {vhi_min_reg} max {vhi_max_reg}")
    # print(df2.head())  # Для перегляду перших кількох рядків фільтрованого датафрейму

In [ ]:
for i in range(1, 28):
    region_year_analysis(region_id=i, years=("1985", "2010"))

#### Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні

In [ ]:
def drought_years_analysis(df, years_range=("1981", "2024"), threshold_extreme=15, threshold_moderate=(15, 35), percent_threshold=20):
    extreme_drought_years = []
    moderate_drought_years = []

    total_regions = len(set(df['region_id']))
    min_regions_affected = (percent_threshold / 100) * total_regions

    for year in range(int(years_range[0]), int(years_range[1]) + 1):
        yearly_data = df[df['Year'] == year]
        extreme_drought_count = len(yearly_data[yearly_data['VHI'] < threshold_extreme])
        moderate_drought_count = len(yearly_data[(yearly_data['VHI'] >= threshold_moderate[0]) & (yearly_data['VHI'] <= threshold_moderate[1])])

        if extreme_drought_count >= min_regions_affected:
            extreme_drought_years.append(year)
        if moderate_drought_count >= min_regions_affected:
            moderate_drought_years.append(year)

    return extreme_drought_years, moderate_drought_years

In [ ]:
selected_regions = [3, 5, 12]  
selected_years = ("2005", "2015")
# vhi_for_regions_and_years(selected_regions, selected_years)  

In [ ]:
extreme_drought_years, moderate_drought_years = drought_years_analysis(result_df, years_range=("2000", "2010"))
print("Роки з екстремальними посухами:", extreme_drought_years)
print("Роки з помірними посухами:", moderate_drought_years)